In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, clear_output

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")    

In [ ]:
openai = OpenAI()
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash-lite"

gpt_prompt="You are an class teacher of class 8th who teaches science. As today activity " \
"you have to choose topic for discussion with two of your students from your class Harsh and " \
"Prashant. You are very polite and always support students and clears any doubt during conversation."

claude_prompt = "You are a student of class 8th and your name is Harsh. You are very shy and hardly " \
"participate in debates. Science is your favourite subject and you choose to participate today." \
"Listed to the topic and start the conversation with your class mate. Work as a team so that " \
"it helps other student to understand topic. Add real world example to explain the topic. " \
"Continue where your class teacher or another student left."

gemini_prompt = "You are a monitor of clas 8th and your name is Prashant. You are very confident and " \
"always participates in such type of group discussion. You always try to dominate other students in" \
"the group discussion. You ask questions and hardly conveince with others. " \
"Continue where your class teacher or another student left."

gpt_messages = ["Hi class, today we will be having group discussion with Harsh and Prashant "
"on topic photosynthetic"]
claude_messages = []
gemini_messages = []

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    stream = openai.chat.completions.create(model=gpt_model, messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def call_claude():
    messages = [{"role": "system", "content": claude_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude})
        messages.append({"role": "user", "content": gemini})
    stream = openai.chat.completions.create(model=gpt_model, messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def call_gemini():
    messages = [{"role": "system", "content": gemini_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": gemini})
    stream = openai.chat.completions.create(model=gpt_model, messages=messages,stream=True)
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
conversation = ""
handle = display(Markdown(""), display_id=True)

for i in range(5):

    # ===== GPT =====
    gpt_next = ""
    for chunk in call_gpt():
        gpt_next = chunk
        handle.update(Markdown(conversation + f"\n### Class Teacher:\n{gpt_next}"))

    gpt_messages.append(gpt_next)
    conversation += f"\n### Class Teacher:\n{gpt_next}\n"


    # ===== Claude =====
    claude_next = ""
    for chunk in call_claude():
        claude_next = chunk
        handle.update(Markdown(conversation + f"\n### Harsh:\n{claude_next}"))

    claude_messages.append(claude_next)
    conversation += f"\n### Harsh:\n{claude_next}\n"


    # ===== Gemini =====
    gemini_next = ""
    for chunk in call_gemini():
        gemini_next = chunk
        handle.update(Markdown(conversation + f"\n### Prashant:\n{gemini_next}"))

    gemini_messages.append(gemini_next)
    conversation += f"\n### Prashant:\n{gemini_next}\n"